## Tutorial #1: Loading Data

#### This tutorial focuses on loading EEG data in several different ways.  

In [1]:
import mne #open-source Python package for exploring, visualizing, and analyzing human neurophysiological data

EEG data can be kept in 3 different types: raw data, epoched data and evoked (averaged) data.

1. Raw Data: Continues data is stored in raw object in MNE. A raw object has 2D array structure as channels×samples.
   In order to load continues data mne.io.read_raw_fif() function can be used.
   
   !!! Find raw data and convert it to epoched version as an example!!
   
2. Epoched Data: Collection of time-locked trials. It is stored as events×channles×times.

3. Evoked Data: When epoched data is averaged out over trials, evoked data is produced. The output is time-locked to the event. It is stored as channels×times.

In [8]:
## Loading Epoched data

# the file name should end with'epo' to be able to load it via mne
data_file = '817_1_PDDys_ODDBALL_Clean_curated-epo'

# We read the EEG epochs:
epochs = mne.read_epochs(data_file + '.fif')

Reading 817_1_PDDys_ODDBALL_Clean_curated-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...     500.00 ms
        0 CTF compensation matrices available
189 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated


In [47]:
#The following line can be useful for getting general information about the data
epochs.info

<Info | 19 non-empty fields
    bads : list | 0 items
    ch_names : list | Fp1, Fz, F3, F7, FC5, FC1, C3, T7, CP5, ...
    chs : list | 60 items (EEG: 60)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : Transform | 3 items
    dig : list | 60 items (60 EEG)
    events : list | 0 items
    file_id : dict | 4 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 250.0 Hz
    meas_date : NoneType | unspecified
    meas_id : dict | 4 items
    nchan : int | 60
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 500.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    experimenter : NoneType
    gantry_angle : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    proj_id : NoneType
    proj_name : NoneType
    subject_info : NoneType
    xp

In [54]:
## Accessing to the data

# The data can be accessed via:
epochs_data_1 = epochs._data
#or 
epochs_data_2 = epochs.get_data() # because I couldn't find the version above in the MNE documentation
#To check whether the two different ways returns the same data
if epochs_data_1.all() == epochs_data_2.all():
    print("Same data!")

Same data!


In [64]:
# To get names of all the conditions in given data and number of events in each condition
epochs.event_id

{'Standard': 201, 'Target': 200, 'Novel': 202}

In [68]:
#To access epochs with a specific condition
novels = epochs['Novel']

Start time before the event -0.1
Stop time after the event 0.5


In [69]:
tmin = epochs.tmin
tmax = epochs.tmax

print('Start time before the event' , tmin)
print('Stop time after the event' , tmax)

Start time before the event -0.1
Stop time after the event 0.5
